<!-- omit in toc -->
# LangChain AI 代理与上下文工程

上下文工程（Context Engineering）意味着在赋予 AI 任务之前，为其创建一个合适的“工作台”。这个工作台包括：

*   **指令 (Instructions):** AI 应如何行动的指南，例如，扮演一个乐于助人的经济型旅行向导。
*   **知识 (Knowledge):** 从数据库、文档或实时来源获取的有用信息。
*   **记忆 (Memory):** 记住过去的对话，以避免重复或遗忘。
*   **工具 (Tools):** AI 可以使用的功能，如计算器或搜索功能。
*   **偏好 (Preferences):** 关于您的重要细节，如您的偏好或位置。

![Context Engineering](https://cdn-images-1.medium.com/max/1500/1*sCTOzjG6KP7slQuxLZUtNg.png)
*上下文工程 (来源: [LangChain](https://blog.langchain.com/context-engineering-for-agents/) 与 [12Factor](https://github.com/humanlayer/12-factor-agents/tree/main))*

AI 工程师们正从提示工程转向上下文工程，因为……

> 上下文工程专注于为 AI 提供正确的背景和工具，使其回答更智能、更有用。

在本 Notebook 中，我们将探讨如何使用 **LangChain** 和 **LangGraph** 这两个强大的工具来构建 AI 代理、RAG 应用和 LLM 应用，并有效地实施**上下文工程**以提升我们 AI 代理的性能。

### 目录
- [什么是上下文工程?](#what-is-context-engineering)
- [写入上下文: 暂存盘与记忆](#writing-context-scratchpad-and-memory)
- [选择上下文: 状态、记忆、RAG 与工具](#selecting-context-state-memory-rag-and-tools)
- [压缩上下文: 摘要策略](#compressing-context-summarization-strategies)
- [隔离上下文: 子代理与沙箱](#isolating-context-sub-agents-and-sandboxing)
- [总结](#summarizing-everything)

### 什么是上下文工程?
LLM 的工作方式就像一种新型的操作系统。LLM 扮演着 CPU 的角色，而其上下文窗口则像 RAM，作为其短期记忆。但是，和 RAM 一样，上下文窗口的空间有限，无法容纳所有信息。

> 正如操作系统决定什么内容进入 RAM，“上下文工程”就是选择 LLM 应该在其上下文中保留什么内容。

![Different Context Types](https://cdn-images-1.medium.com/max/1000/1*kMEQSslFkhLiuJS8-WEMIg.png)

在构建 LLM 应用时，我们需要管理不同类型的上下文。上下文工程涵盖以下主要类型：

*   指令: 提示、示例、记忆和工具描述
*   知识: 事实、存储的信息和记忆
*   工具: 工具调用的反馈和结果

今年，人们对代理越来越感兴趣，因为 LLM 在思考和使用工具方面表现得更好。代理通过结合 LLM 和工具来处理长期任务，并根据工具的反馈选择下一步行动。

![Agent Workflow](https://cdn-images-1.medium.com/max/1500/1*Do44CZkpPYyIJefuNQ69GA.png)

但是，长期任务和从工具收集过多反馈会消耗大量 Token。这可能导致问题：上下文窗口溢出、成本和延迟增加，以及代理性能下降。

Drew Breunig 解释了过多的上下文如何损害性能，包括：

*   上下文污染 (Context Poisoning): [当错误或幻觉被添加到上下文中时](https://www.dbreunig.com/2025/06/22/how-contexts-fail-and-how-to-fix-them.html?ref=blog.langchain.com#context-poisoning)
*   上下文分心 (Context Distraction): [当过多的上下文使模型困惑时](https://www.dbreunig.com/2025/06/22/how-contexts-fail-and-how-to-fix-them.html?ref=blog.langchain.com#context-distraction)
*   上下文混淆 (Context Confusion): [当额外、不必要的细节影响答案时](https://www.dbreunig.com/2025/06/22/how-contexts-fail-and-how-to-fix-them.html?ref=blog.langchain.com#context-confusion)
*   上下文冲突 (Context Clash): [当部分上下文提供矛盾信息时](https://www.dbreunig.com/2025/06/22/how-contexts-fail-and-how-to-fix-them.html?ref=blog.langchain.com#context-clash)

![Multiple turns in Agent](https://cdn-images-1.medium.com/max/1500/1*ZJeZJPKI5jC_1BMCoghZxA.png)

Anthropic 在他们的研究中强调了这一点：

> 代理的对话通常有数百轮，因此仔细管理上下文至关重要。

那么，如今人们如何解决这个问题？代理上下文工程的常见策略可分为四种主要类型：

*   **写入 (Write)**: 创建清晰有用的上下文
*   **选择 (Select)**: 只挑选最相关的信息
*   **压缩 (Compress)**: 缩短上下文以节省空间
*   **隔离 (Isolate)**: 将不同类型的上下文分开

![Categories of Context Engineering](https://cdn-images-1.medium.com/max/2600/1*CacnXVAI6wR4eSIWgnZ9sg.png)
*上下文工程的类别 (来源: [LangChain 文档](https://blog.langchain.com/context-engineering-for-agents/))*

[LangGraph](https://www.langchain.com/langgraph) 被设计用来支持所有这些策略。我们将逐一介绍这些组件，并了解它们如何帮助我们的 AI 代理更好地工作。

### 写入上下文: 暂存盘与记忆

上下文工程的第一个原则是**写入**上下文。这意味着在 LLM 的直接上下文窗口之外创建和存储信息，代理可以在之后访问这些信息。我们将探讨在 LangGraph 中实现这一点的两种主要机制：**暂存盘 (Scratchpad)**（用于短期的、特定于会话的笔记）和**记忆 (Memory)**（用于跨会话的长期持久化）。

![First Component of CE](https://cdn-images-1.medium.com/max/1000/1*aXpKxYt03iZPcrGkxsFvrQ.png)

#### 使用 LangGraph 实现暂存盘
就像人类做笔记以备后续任务一样，代理可以使用[暂存盘](https://www.anthropic.com/engineering/claude-think-tool)来做同样的事情。它将信息存储在上下文窗口之外，以便代理在需要时可以访问。

一个很好的例子是 [Anthropic 的多代理研究员](https://www.anthropic.com/engineering/built-multi-agent-research-system)：

> *首席研究员规划其方法并将其保存到记忆中，因为如果上下文窗口超过 200,000 个 Token，它将被截断，因此保存计划可确保其不会丢失。*

在 LangGraph 中，`StateGraph` 对象充当此暂存盘。状态是在图的节点之间传递的核心数据结构。您可以定义其模式，每个节点都可以读取和写入它。这为您的代理维护短期的、线程范围的记忆提供了一种强大的方式。

首先，让我们设置我们的环境和用于打印的辅助工具。

In [ ]:
# 导入必要的库，用于类型提示、格式化和环境管理
import getpass
import os
from typing import TypedDict

from IPython.display import Image, display
from rich.console import Console
from rich.pretty import pprint

# 初始化一个控制台，用于在 Notebook 中进行富文本格式化输出。
console = Console()

# 设置 Anthropic API 密钥以验证请求
# 出于安全考虑，建议将其设置为环境变量
if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("请提供您的 Anthropic API 密钥: ")

接下来，我们将为状态对象创建一个 `TypedDict`。这定义了我们暂存盘的模式，确保数据在流经图时的一致性。

In [ ]:
# 使用 TypedDict 定义图状态的模式。
# 这个类作为在图的节点之间传递的数据结构。
# 它确保状态具有一致的形状并提供类型提示。
class State(TypedDict):
    """
    定义我们的笑话生成器工作流的状态结构。

    属性:
        topic: 将为其生成笑话的输入主题。
        joke: 将存储生成的笑话的输出字段。
    """

    topic: str
    joke: str

#### 创建一个 StateGraph 以写入暂存盘
一旦我们定义了一个状态对象，我们就可以使用 `StateGraph` 将上下文写入其中。StateGraph 是 LangGraph 用于构建有状态代理的主要工具。

- **节点** 是工作流中的步骤。每个节点都是一个函数，它以当前状态作为输入并返回更新。
- **边** 连接节点，定义执行流程。

让我们创建一个聊天模型和一个节点函数，该函数使用它来生成一个笑话并将其写入我们的状态对象。

In [ ]:
# 导入 LangChain 和 LangGraph 的必要库
from langchain.chat_models import init_chat_model
from langgraph.graph import END, START, StateGraph

# --- 模型设置 ---
# 初始化将在工作流中使用的聊天模型
# 我们使用一个特定的 Claude 模型，温度为 0 以获得确定性输出
llm = init_chat_model("anthropic:claude-3-sonnet-20240229", temperature=0)

# --- 定义工作流节点 ---
def generate_joke(state: State) -> dict[str, str]:
    """
    一个节点函数，根据当前状态中的主题生成一个笑话。

    此函数从状态中读取 'topic'，使用 LLM 生成一个笑话，
    并返回一个字典以更新状态中的 'joke' 字段。

    参数:
        state: 图的当前状态，必须包含一个 'topic'。

    返回:
        一个带有 'joke' 键的字典以更新状态。
    """
    # 从状态中读取主题
    topic = state["topic"]
    print(f"正在生成一个关于 {topic} 的笑话...")

    # 调用语言模型生成一个笑话
    msg = llm.invoke(f"写一个关于 {topic} 的短笑话")

    # 返回生成的笑话以写回状态
    return {"joke": msg.content}

# --- 构建和编译图 ---
# 使用预定义的状态模式初始化一个新的 StateGraph
workflow = StateGraph(State)

# 将 'generate_joke' 函数作为图中的一个节点添加
workflow.add_node("generate_joke", generate_joke)

# 定义工作流的执行路径：
# 图从 START 入口点开始，流向我们的 'generate_joke' 节点。
workflow.add_edge(START, "generate_joke")
# 'generate_joke' 完成后，图执行结束。
workflow.add_edge("generate_joke", END)

# 将工作流编译成一个可执行的链
chain = workflow.compile()

# --- 可视化图 ---
# 显示已编译工作流图的可视化表示
display(Image(chain.get_graph().draw_mermaid_png()))

现在我们可以执行这个工作流。它将接受一个带有 `topic` 的初始状态，运行 `generate_joke` 节点，并将结果写入状态的 `joke` 字段。

In [ ]:
# --- 执行工作流 ---
# 使用包含主题的初始状态调用已编译的图。
# `invoke` 方法从 START 节点运行到 END 节点。
joke_generator_state = chain.invoke({"topic": "cats"})

# --- 显示最终状态 ---
# 打印执行后图的最终状态。
# 这将显示写入状态的输入 'topic' 和输出 'joke'。
console.print("\n[bold blue]笑话生成器最终状态:[/bold blue]")
pprint(joke_generator_state)

#### 在 LangGraph 中写入记忆
暂存盘帮助代理在单个会话中工作，但有时代理需要跨多个会话记住事情。这就是长期记忆发挥作用的地方。

*   [Reflexion](https://arxiv.org/abs/2303.11366) 引入了代理在每轮后进行反思并重用自生成提示的想法。
*   [Generative Agents](https://ar5iv.labs.arxiv.org/html/2304.03442) 通过总结过去的代理反馈来创建长期记忆。

![Memory Writing](https://cdn-images-1.medium.com/max/1000/1*VaMVevdSVxDITLK1j0LfRQ.png)

LangGraph 通过可以传递给已编译图的 `store` 来支持长期记忆。这允许您在*线程*（例如，不同的聊天会话）之间持久化上下文。

- **检查点 (Checkpointing)** 在 `thread` 的每个步骤保存图的状态。
- **长期记忆** 允许您使用键值 `BaseStore` 在线程之间保留特定上下文。

让我们增强我们的代理以使用短期检查点和长期记忆存储。

In [ ]:
# 从 LangGraph 导入记忆和持久化组件
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore

# 初始化存储组件
checkpointer = InMemorySaver()  # 用于线程级状态持久化（短期记忆）
memory_store = InMemoryStore()  # 用于跨线程记忆存储（长期记忆）

# 定义一个命名空间以在长期存储中逻辑地分组相关数据。
namespace = ("rlm", "joke_generator")

def generate_joke_with_memory(state: State, store: BaseStore) -> dict[str, str]:
    """生成具有记忆意识的笑话。
    
    这个增强版本在生成新笑话之前会检查长期记忆中是否存在现有笑话，并保存新笑话。
    
    参数:
        state: 包含主题的当前状态。
        store: 用于持久化上下文的记忆存储。
        
    返回:
        一个包含生成的笑话的字典。
    """
    # 检查记忆中是否存在现有笑话（我们稍后会介绍选择）
    existing_jokes = list(store.search(namespace))
    if existing_jokes:
        existing_joke_content = existing_jokes[0].value
        print(f"在记忆中找到现有笑话: {existing_joke_content}")
    else:
        print("在记忆中未找到现有笑话。")

    # 根据主题生成一个新笑话
    msg = llm.invoke(f"写一个关于 {state['topic']} 的短笑话")
    
    # 将新笑话写入长期记忆
    store.put(namespace, "last_joke", {"joke": msg.content})
    print(f"已将新笑话写入记忆: {msg.content[:50]}...")

    # 返回笑话以添加到当前会话的状态（暂存盘）
    return {"joke": msg.content}


# 构建具有记忆功能的工作流
workflow_with_memory = StateGraph(State)
workflow_with_memory.add_node("generate_joke", generate_joke_with_memory)
workflow_with_memory.add_edge(START, "generate_joke")
workflow_with_memory.add_edge("generate_joke", END)

# 使用检查点（用于会话状态）和记忆存储（用于长期）进行编译
chain_with_memory = workflow_with_memory.compile(checkpointer=checkpointer, store=memory_store)

现在，让我们执行更新后的工作流。我们将使用一个 `config` 对象来指定一个 `thread_id`。这标识了当前会话。我们第一次运行它时，长期记忆中应该没有笑话。

In [ ]:
# 在特定线程（例如，一个用户会话）内执行工作流
config_thread_1 = {"configurable": {"thread_id": "1"}}joke_state_thread_1 = chain_with_memory.invoke({"topic": "dogs"}, config_thread_1)

# 显示第一个线程的工作流结果
console.print("\n[bold cyan]工作流结果 (线程 1):[/bold cyan]")
pprint(joke_state_thread_1)

因为我们使用检查点编译了工作流，所以我们现在可以查看该线程的图的最新状态。这显示了短期暂存盘的价值。

In [ ]:
# --- 检索和检查图状态 ---
# 使用 `get_state` 方法检索线程 "1" 的最新状态快照。
latest_state_thread_1 = chain_with_memory.get_state(config_thread_1)

# --- 显示状态快照 ---
# StateSnapshot 不仅包括数据（'topic', 'joke'），还包括执行元数据。
console.print("\n[bold magenta]最新图状态 (线程 1):[/bold magenta]")
pprint(latest_state_thread_1)

现在，让我们再次运行工作流，但使用一个*不同*的 `thread_id`。这模拟了一个新的会话。我们的长期记忆存储现在应该包含来自第一个会话的笑话，展示了上下文如何可以被持久化并在线程之间共享。

In [ ]:
# 使用不同的线程 ID 执行工作流以模拟新会话
config_thread_2 = {"configurable": {"thread_id": "2"}}joke_state_thread_2 = chain_with_memory.invoke({"topic": "birds"}, config_thread_2)

# 显示结果，它应该显示在记忆中找到了来自前一个线程的笑话
console.print("\n[bold yellow]工作流结果 (线程 2):[/bold yellow]")
pprint(joke_state_thread_2)

### 选择上下文: 状态、记忆、RAG 与工具

第二个原则是**选择**上下文。一旦上下文被写入，代理需要能够为当前任务检索*最相关*的信息片段。这可以防止上下文窗口溢出并使代理保持专注。

![Second Component of CE](https://cdn-images-1.medium.com/max/1000/1*VZiHtQ_8AlNdV3HIMrbBZA.png)

我们将探讨四种选择上下文的方式：
1.  **从暂存盘（状态）中选择:** 选择在当前会话中写入的数据。
2.  **从长期记忆中选择:** 从过去的会话中检索数据。
3.  **从知识（RAG）中选择:** 使用检索增强生成从文档中获取信息。
4.  **从工具（Tool-RAG）中选择:** 使用 RAG 为任务选择最佳工具。

#### 暂存盘选择方法
您如何从暂存盘中选择上下文取决于其实现方式。由于我们的暂存盘是代理的运行时 `State` 对象，我们（开发者）决定在每个步骤中与代理共享状态的哪些部分。这提供了细粒度的控制。

让我们创建一个两步工作流。第一个节点生成一个笑话（写入状态）。第二个节点从状态中*选择*该笑话并加以改进。

In [ ]:
# 我们需要一个可以容纳原始笑话和改进后笑话的状态
class JokeImprovementState(TypedDict):
    topic: str
    joke: str
    improved_joke: str

def improve_joke(state: JokeImprovementState) -> dict[str, str]:
    """通过添加文字游戏来改进现有笑话。
    
    这演示了从状态中选择上下文 - 我们从状态中读取现有笑话并用它来生成一个改进版本。
    
    参数:
        state: 包含原始笑话的当前状态。
        
    返回:
        一个包含改进后笑话的字典。
    """
    initial_joke = state["joke"]
    print(f"从状态中选择的初始笑话: {initial_joke[:50]}...")
    
    # 从状态中选择笑话以呈现给 LLM
    msg = llm.invoke(f"通过添加文字游戏使这个笑话更有趣: {initial_joke}")
    return {"improved_joke": msg.content}

# --- 构建两步工作流 ---
selection_workflow = StateGraph(JokeImprovementState)

# 添加初始笑话生成节点（重用之前的）
selection_workflow.add_node("generate_joke", generate_joke)
# 添加新的改进节点
selection_workflow.add_node("improve_joke", improve_joke)

# 按顺序连接节点
selection_workflow.add_edge(START, "generate_joke")
selection_workflow.add_edge("generate_joke", "improve_joke")
selection_workflow.add_edge("improve_joke", END)

# 编译工作流
selection_chain = selection_workflow.compile()

# 可视化新图
display(Image(selection_chain.get_graph().draw_mermaid_png()))

In [ ]:
# 执行工作流以查看上下文选择的实际效果
joke_improvement_state = selection_chain.invoke({"topic": "computers"})

# 使用富文本格式显示最终状态
console.print("\n[bold blue]最终笑话改进状态:[/bold blue]")
pprint(joke_improvement_state)

#### 记忆选择能力
如果代理可以保存记忆，它们也需要为手头的任务选择相关的记忆。这对于回忆以下内容很有用：
- **情景记忆:** 所需行为的少量示例。
- **程序记忆:** 指导行为的指令。
- **语义记忆:** 提供与任务相关的上下文的事实或关系。

在我们之前的示例中，我们写入了 `InMemoryStore`。现在，我们可以使用 `store.get()` 方法从中选择上下文，以将相关状态拉入我们的工作流。让我们创建一个节点，该节点选择先前存储的笑话并尝试生成一个*不同*的笑话。

In [ ]:
# 为此示例重新初始化存储组件
checkpointer_select = InMemorySaver()
memory_store_select = InMemoryStore()
# 用一个笑话预填充存储以供选择
memory_store_select.put(namespace, "last_joke", {"joke": "为什么电脑会冷？因为它把窗户开着！"})

def generate_different_joke(state: State, store: BaseStore) -> dict[str, str]:
    """生成具有记忆感知上下文选择的笑话。
    
    此函数演示了在生成新内容之前从记忆中选择上下文，确保它不会重复。
    
    参数:
        state: 包含主题的当前状态
        store: 用于持久化上下文的记忆存储
        
    返回:
        一个包含新生成的笑话的字典
    """
    # 如果存在，则从记忆中选择先前的笑话
    prior_joke_item = store.get(namespace, "last_joke")
    prior_joke_text = "None"
    if prior_joke_item:
        prior_joke_text = prior_joke_item.value["joke"]
        print(f"从记忆中选择的先前笑话: {prior_joke_text}")
    else:
        print("在记忆中未找到先前的笑话。")

    # 生成一个与先前笑话不同的新笑话
    prompt = (
        f"写一个关于 {state['topic']} 的短笑话， "
        f"但要与这个先前的笑话不同: '{prior_joke_text}'"
    )
    msg = llm.invoke(prompt)

    # 将新笑话存储在记忆中以备将来上下文选择
    store.put(namespace, "last_joke", {"joke": msg.content})

    return {"joke": msg.content}

# 构建具有记忆感知的工作流
memory_selection_workflow = StateGraph(State)
memory_selection_workflow.add_node("generate_joke", generate_different_joke)
memory_selection_workflow.add_edge(START, "generate_joke")
memory_selection_workflow.add_edge("generate_joke", END)

# 使用检查点和记忆存储进行编译
memory_selection_chain = memory_selection_workflow.compile(checkpointer=checkpointer_select, store=memory_store_select)

# 执行工作流
config = {"configurable": {"thread_id": "3"}}new_joke_state = memory_selection_chain.invoke({"topic": "computers"}, config)

console.print("\n[bold green]记忆选择工作流最终状态:[/bold green]")
pprint(new_joke_state)

#### LangGraph BigTool 调用的优势（工具选择）
代理使用工具，但给它们太多工具可能会导致混淆，尤其是在工具描述重叠时。一种解决方案是对工具描述使用 RAG，以获取与任务最相关的工具。

> 根据[最近的研究](https://arxiv.org/abs/2505.03275)，这可以将工具选择的准确性提高多达 3 倍。

`langgraph-bigtool` 库非常适合此目的。它对工具描述应用语义相似性搜索以选择最相关的工具。让我们通过创建一个包含 Python 内置 `math` 库中所有函数的代理来演示，并了解它如何选择正确的函数。

In [ ]:
# 导入此示例所需的库
import math
import types
import uuid

from langchain.embeddings import init_embeddings
from langgraph_bigtool import create_agent
from langgraph_bigtool.utils import convert_positional_only_function_to_tool
from utils import format_messages # 从提供的 utils.py 中导入的辅助函数

# 确保为嵌入设置了 OpenAI API 密钥
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("请提供您的 OpenAI API 密钥: ")

# --- 1. 收集和准备工具 ---
# 从 `math` 模块收集所有内置函数
all_math_tools = []
for function_name in dir(math):
    function = getattr(math, function_name)
    if isinstance(function, types.BuiltinFunctionType):
        # 这处理了 `math` 库函数签名的特殊性
        if tool := convert_positional_only_function_to_tool(function):
            all_math_tools.append(tool)

# 创建一个将唯一 ID 映射到每个工具实例的注册表
tool_registry = {str(uuid.uuid4()): tool for tool in all_math_tools}

# --- 2. 索引工具以进行语义搜索 ---
# 初始化嵌入模型
embeddings = init_embeddings("openai:text-embedding-3-small")

# 设置一个配置为对工具描述进行向量搜索的内存中存储
tool_store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536, # text-embedding-3-small 的维度
        "fields": ["description"],
    }
)

# 将每个工具的名称和描述索引到存储中
for tool_id, tool in tool_registry.items():
    tool_store.put(
        ("tools",), # 工具的命名空间
        tool_id,
        {"description": f"{tool.name}: {tool.description}"}
    )

# --- 3. 创建和编译代理 ---
# langgraph-bigtool 中的 create_agent 函数设置代理逻辑
builder = create_agent(llm, tool_registry)
bigtool_agent = builder.compile(store=tool_store)

display(Image(bigtool_agent.get_graph().draw_mermaid_png()))

In [ ]:
# --- 4. 调用代理 ---
# 定义代理的查询。这需要选择正确的数学工具。
query = "使用可用工具计算 0.5 的反余弦。"

# 调用代理。它将首先搜索其工具，选择 'acos'，然后执行它。
result = bigtool_agent.invoke({"messages": query})

# 格式化并显示代理执行的最终消息。
# 输出将显示代理的思考过程：搜索、查找和使用工具。
format_messages(result['messages'])

#### 使用上下文工程进行 RAG（知识选择）
[RAG（检索增强生成）](https://github.com/langchain-ai/rag-from-scratch) 是上下文工程的基石。它允许代理从庞大的文档库中选择相关知识。

在 LangGraph 中，这通常通过创建一个检索工具来完成。让我们构建一个可以回答有关 Lilian Weng 博客文章问题的 RAG 代理。

In [ ]:
# 导入 RAG 所需的组件
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage, ToolMessage
from typing_extensions import Literal

# --- 1. 加载和分块文档 ---
# 定义 Lilian Weng 博客文章的 URL
urls = [
    "https://lilianweng.github.io/posts/2025-05-01-thinking/",
    "https://lilianweng.github.io/posts/2024-11-28-reward-hacking/",
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2024-04-12-diffusion-video/",
]
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# 将文档分割成更小的块以便有效检索
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)

# --- 2. 创建向量存储和检索工具 ---
vectorstore = InMemoryVectorStore.from_documents(documents=doc_splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

# 创建一个代理可以调用的检索工具
retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "搜索并返回有关 Lilian Weng 博客文章的信息。",
)

rag_tools = [retriever_tool]
rag_tools_by_name = {tool.name: tool for tool in rag_tools}
llm_with_rag_tools = llm.bind_tools(rag_tools)

现在我们为我们的 RAG 代理定义图组件：提示、用于调用 LLM 和工具的节点，以及一个用于创建循环的条件边。

In [ ]:
# --- 3. 定义 RAG 代理图 ---
rag_prompt = """您是一个有用的助手，负责从 Lilian Weng 的一系列技术博客文章中检索信息。 
在使用检索工具收集上下文之前，请与用户澄清研究范围。反思您获取的任何上下文，并
继续，直到您有足够的上下文来回答用户的研究请求。"""

def rag_llm_call(state: MessagesState):
    """调用 LLM 的节点。LLM 决定是调用工具还是生成最终答案。"""
    messages_with_prompt = [SystemMessage(content=rag_prompt)] + state["messages"]
    response = llm_with_rag_tools.invoke(messages_with_prompt)
    return {"messages": [response]}

def rag_tool_node(state: dict):
    """执行工具调用并返回观察结果的节点。"""
    last_message = state["messages"][-1]
    result = []
    for tool_call in last_message.tool_calls:
        tool = rag_tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(content=str(observation), tool_call_id=tool_call["id"]))
    return {"messages": result}

def should_continue_rag(state: MessagesState) -> Literal["Action", END]:
    """决定下一步的条件边。如果 LLM 进行了工具调用，则路由到工具节点。否则，结束。"""
    if state["messages"][-1].tool_calls:
        return "Action"
    return END

# 构建 RAG 代理工作流
rag_agent_builder = StateGraph(MessagesState)
rag_agent_builder.add_node("llm_call", rag_llm_call)
rag_agent_builder.add_node("Action", rag_tool_node)
rag_agent_builder.set_entry_point("llm_call")
rag_agent_builder.add_conditional_edges("llm_call", should_continue_rag, {"Action": "Action", END: END})
rag_agent_builder.add_edge("Action", "llm_call")

rag_agent = rag_agent_builder.compile()
display(Image(rag_agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# --- 4. 调用 RAG 代理 ---
query = "博客中讨论的奖励黑客类型有哪些？"
result = rag_agent.invoke({"messages": [("user", query)]})
format_messages(result['messages'])

### 压缩上下文: 摘要策略

第三个原则是**压缩**上下文。代理交互可能跨越数百轮，并涉及消耗大量 Token 的工具调用。摘要是管理此问题的常见且有效的方法，可以在保留基本信息的同时减少 Token 数量。

![Third Component of CE](https://cdn-images-1.medium.com/max/1000/1*Xu76qgF1u2G3JipeIgHo5Q.png)

我们可以在代理工作流的不同点添加摘要：
- 在对话结束时，创建整个交互的摘要。
- 在消耗大量 Token 的工具调用之后，压缩其输出，然后再进入代理的暂存盘。

让我们探讨这两种方法。

#### 方法 1: 总结整个对话

首先，我们将构建一个代理，该代理执行其 RAG 任务，然后作为最后一步，生成整个交互的摘要。这对于日志记录或创建代理工作的简洁记录非常有用。

In [ ]:
from rich.markdown import Markdown

# 定义一个包含摘要字段的扩展状态
class StateWithSummary(MessagesState):
    summary: str

summarization_prompt = """总结完整的聊天记录和所有工具反馈，以概述用户询问的内容以及代理所做的工作。"""

def summary_node(state: MessagesState) -> dict:
    """生成对话摘要的节点。"""
    messages = [SystemMessage(content=summarization_prompt)] + state["messages"]
    result = llm.invoke(messages)
    return {"summary": result.content}

def should_continue_to_summary(state: MessagesState) -> Literal["Action", "summary_node"]:
    """决定路由到工具操作还是最终摘要节点的条件边。"""
    if state["messages"][-1].tool_calls:
        return "Action"
    return "summary_node"

# 构建带有最终摘要步骤的工作流
summary_agent_builder = StateGraph(StateWithSummary)
summary_agent_builder.add_node("llm_call", rag_llm_call)
summary_agent_builder.add_node("Action", rag_tool_node)
summary_agent_builder.add_node("summary_node", summary_node)
summary_agent_builder.set_entry_point("llm_call")
summary_agent_builder.add_conditional_edges("llm_call", should_continue_to_summary, {"Action": "Action", "summary_node": "summary_node"})
summary_agent_builder.add_edge("Action", "llm_call")
summary_agent_builder.add_edge("summary_node", END)

summary_agent = summary_agent_builder.compile()
display(Image(summary_agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# 运行代理并显示最终摘要
query = "根据博客，为什么 RL 能改善 LLM 的推理能力？"
result = summary_agent.invoke({"messages": [("user", query)]})

console.print("\n[bold green]最终代理消息:[/bold green]")
format_messages([result['messages'][-1]])

console.print("\n[bold purple]生成的对话摘要:[/bold purple]")
display(Markdown(result["summary"]))

**注意:** 虽然有效，但这种方法可能会消耗大量 Token，因为完整的、未压缩的工具输出会通过循环传递。对于上面的查询，这可能会使用超过 10 万个 Token。

#### 方法 2: 动态压缩工具输出
一种更有效的方法是在上下文进入代理的主要暂存盘*之前*对其进行压缩。让我们更新 RAG 代理，以便在收到工具调用输出后立即对其进行总结。

In [ ]:
tool_summarization_prompt = """您将收到来自 RAG 系统的文档。
总结该文档，确保保留所有相关和基本信息。
您的目标是减少文档的大小（Token），使其更易于代理管理。"""

def tool_node_with_summarization(state: dict):
    """执行工具调用，然后立即总结输出。"""
    last_message = state["messages"][-1]
    result = []
    for tool_call in last_message.tool_calls:
        tool = rag_tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        
        # 在将文档添加到状态之前对其进行总结
        summary_msg = llm.invoke([
            SystemMessage(content=tool_summarization_prompt),
            ("user", str(observation))
        ])
        
        result.append(ToolMessage(content=summary_msg.content, tool_call_id=tool_call["id"]))
    return {"messages": result}

# 构建更高效的工作流
efficient_agent_builder = StateGraph(MessagesState)
efficient_agent_builder.add_node("llm_call", rag_llm_call)
efficient_agent_builder.add_node("Action", tool_node_with_summarization)
efficient_agent_builder.set_entry_point("llm_call")
efficient_agent_builder.add_conditional_edges("llm_call", should_continue_rag, {"Action": "Action", END: END})
efficient_agent_builder.add_edge("Action", "llm_call")

efficient_agent = efficient_agent_builder.compile()
display(Image(efficient_agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# 使用高效代理运行相同的查询
query = "根据博客，为什么 RL 能改善 LLM 的推理能力？"
result = efficient_agent.invoke({"messages": [("user", query)]})

console.print("\n[bold green]高效代理对话流:[/bold green]")
format_messages(result['messages'])

**结果:** 这个简单的更改可以将 Token 使用量减少近一半，使代理的效率和成本效益大大提高，展示了动态上下文压缩的强大功能。

### 隔离上下文: 子代理与沙箱
最后一个原则是**隔离**上下文。这涉及分割上下文，以防止不同任务或类型的信息相互干扰。这对于复杂的多步骤问题至关重要。

![Fourth Component of CE](https://cdn-images-1.medium.com/max/1000/1*-b9BLPkLHkYsy2iLQIdxUg.png)

我们将探讨两种强大的隔离技术：
1.  **子代理架构:** 使用由主管管理的多个专业代理。
2.  **沙箱环境:** 在安全的隔离环境中执行代码。

#### 使用子代理架构隔离上下文

一种常见的隔离上下文的方法是在子代理之间分配任务。OpenAI 的 [Swarm](https://github.com/openai/swarm) 库就是为此“关注点分离”而设计的，其中每个代理管理一个特定的子任务，拥有自己的工具、指令和上下文窗口。

> *子代理在其自己的上下文窗口中并行操作，同时探索问题的不同方面。* - Anthropic

LangGraph 通过**主管**架构支持这一点。主管将任务委派给专业的子代理，每个子代理都在其自己的隔离上下文窗口中运行。让我们构建一个管理 `math_expert` 和 `research_expert` 的主管。

In [ ]:
# 导入预构建的代理创建器
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

# --- 1. 为每个代理定义工具 ---
def add(a: float, b: float) -> float:
    """将两个数字相加。"""
    return a + b

def multiply(a: float, b: float) -> float:
    """将两个数字相乘。"""
    return a * b

def web_search(query: str) -> str:
    """模拟网络搜索函数，返回 FAANG 公司的人数。"""
    return (
        "以下是 2024 年各 FAANG 公司的员工人数：\n"
        "1. **Facebook (Meta)**: 67,317 名员工。\n"
        "2. **Apple**: 164,000 名员工。\n"
        "3. **Amazon**: 1,551,000 名员工。\n"
        "4. **Netflix**: 14,000 名员工。\n"
        "5. **Google (Alphabet)**: 181,269 名员工。"
    )

# --- 2. 创建专业代理 ---
# 每个代理都有自己的工具和指令，隔离其上下文
math_agent = create_react_agent(
    model=llm,
    tools=[add, multiply],
    name="math_expert",
    prompt="您是一位数学专家。一次只使用一个工具。"
)

research_agent = create_react_agent(
    model=llm,
    tools=[web_search],
    name="research_expert",
    prompt="您是一位世界一流的研究员，可以访问网络搜索。不要进行任何数学计算。"
)

# --- 3. 创建主管工作流 ---
# 主管协调代理
supervisor_workflow = create_supervisor(
    [research_agent, math_agent],
    model=llm,
    prompt=(
        "您是一个团队主管，管理一名研究专家和一名数学专家。 "
        "将任务委派给适当的代理以回答用户的查询。 "
        "对于时事或事实，请使用 research_agent。 "
        "对于数学问题，请使用 math_agent。"
    )
)

# 编译多代理应用程序
multi_agent_app = supervisor_workflow.compile()

In [ ]:
# --- 4. 执行多代理工作流 ---
result = multi_agent_app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "2024 年 FAANG 公司的总员工人数是多少？"
        }
    ]
})

# 格式化并显示结果，显示委派的实际效果
format_messages(result['messages'])

#### 使用沙箱环境进行隔离
另一种强大的隔离上下文的方法是使用沙箱执行环境。`CodeAgent` 可以在安全的沙箱中编写和执行代码，而不是让 LLM 仅通过 JSON 调用工具。然后将结果返回给 LLM。

这将繁重的数据或复杂的状态（如脚本中的变量）保留在 LLM 的 Token 限制之外，将其隔离在环境中。

`langchain-sandbox` 提供了一个使用 Pyodide（编译为 WebAssembly 的 Python）安全执行不受信任的 Python 代码的环境。我们可以将其作为工具添加到任何 LangGraph 代理中。

**注意:** 需要 Deno。从此处安装: https://docs.deno.com/runtime/getting_started/installation/

In [ ]:
# 导入沙箱工具和预构建的代理
from langchain_sandbox import PyodideSandboxTool
from langgraph.prebuilt import create_react_agent

# 创建一个沙箱工具。allow_net=True 允许它在需要时安装软件包。
sandbox_tool = PyodideSandboxTool(allow_net=True)

# 创建一个配备沙箱工具的 ReAct 代理
sandbox_agent = create_react_agent(llm, tools=[sandbox_tool])

# 执行一个代理可以通过编写和运行 Python 代码来解决的查询
result = await sandbox_agent.ainvoke(
    {"messages": [{"role": "user", "content": "5 + 7 是多少？"}]},
)

# 格式化并显示结果
format_messages(result['messages'])

#### LangGraph 中的状态隔离
最后，重要的是要记住，代理的**运行时状态对象**本身就是一种强大的隔离上下文的方式。通过设计具有不同字段的状态模式，您可以控制 LLM 看到的内容。

例如，一个字段（如 `messages`）可以在每轮都显示给 LLM，而其他字段则存储信息（如原始工具输出或中间计算），这些信息在特定节点需要访问之前保持隔离。您在本 Notebook 中已经看到了许多这样的示例，我们明确地从状态对象的特定字段中读取和写入。

### 总结
让我们总结一下到目前为止我们所做的工作：

*   **写入:** 我们使用 LangGraph `StateGraph` 创建了一个**“暂存盘”**用于短期记忆，并使用 `InMemoryStore` 用于长期记忆，使我们的代理能够存储和回忆信息。
*   **选择:** 我们演示了如何从代理的状态和长期记忆中选择性地提取相关信息。这包括使用检索增强生成（`RAG`）来查找特定知识，以及使用 `langgraph-bigtool` 从众多选项中选择正确的工具。
*   **压缩:** 为了管理长对话和消耗大量 Token 的工具输出，我们实施了摘要。我们展示了如何动态压缩 `RAG` 结果，以使代理更高效并减少 Token 使用量。
*   **隔离:** 我们探讨了通过构建一个由主管将任务委派给专业子代理的多代理系统，以及通过使用沙箱环境来运行代码，来保持上下文分离以避免混淆。

所有这些技术都属于**“上下文工程”**——一种通过仔细管理其工作记忆（`context`）来改进 AI 代理的策略，使其更高效、准确，并能够处理复杂的长期任务。